# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
pd.set_option('display.max_columns', None)
import re
from sqlalchemy import create_engine
import sqlite3
import random
import nltk
from collections import defaultdict
import pickle
import cleaner
import augmenter

In [2]:
# load messages dataset
df_mess = pd.read_csv('messages.csv')
df_mess.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# load categories dataset
df_cat = pd.read_csv('categories.csv')
df_cat.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [4]:
# merge datasets
df = df_mess[['id','message']].merge(df_cat, left_on='id', right_on='id')
df.head()

,id,message,categories
0,2,Weather update - a cold front from Cuba that c...,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",related-1;request-0;offer-0;aid_related-0;medi...


In [5]:
# check for NaN gaps
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26386 entries, 0 to 26385
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          26386 non-null  int64 
 1   message     26386 non-null  object
 2   categories  26386 non-null  object
dtypes: int64(1), object(2)
memory usage: 824.6+ KB


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [6]:
# determine list of categories
row = df_cat.categories[0]
category_colnames = [s[:-2] for s in row.split(';')]
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [7]:
# build dataframe of categorys with integer values
df_form = pd.DataFrame()
for i, nrow in df.iterrows():
    df_form[nrow['id']] = [int(s[-1]) for s in df.categories[i].split(';')]
df_form = df_form.transpose().reset_index().rename(columns={'index':'id'})
df_form.columns = ['id'] + category_colnames
df_form.head()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# check for NaN's
df_form.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26180 entries, 0 to 26179
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   id                      26180 non-null  int64
 1   related                 26180 non-null  int64
 2   request                 26180 non-null  int64
 3   offer                   26180 non-null  int64
 4   aid_related             26180 non-null  int64
 5   medical_help            26180 non-null  int64
 6   medical_products        26180 non-null  int64
 7   search_and_rescue       26180 non-null  int64
 8   security                26180 non-null  int64
 9   military                26180 non-null  int64
 10  child_alone             26180 non-null  int64
 11  water                   26180 non-null  int64
 12  food                    26180 non-null  int64
 13  shelter                 26180 non-null  int64
 14  clothing                26180 non-null  int64
 15  money              

In [9]:
# Check if Binary
for c in category_colnames:
    print(c, df_form[c].unique())

related [1 0 2]
request [0 1]
offer [0 1]
aid_related [0 1]
medical_help [0 1]
medical_products [0 1]
search_and_rescue [0 1]
security [0 1]
military [0 1]
child_alone [0]
water [0 1]
food [0 1]
shelter [0 1]
clothing [0 1]
money [0 1]
missing_people [0 1]
refugees [0 1]
death [0 1]
other_aid [0 1]
infrastructure_related [0 1]
transport [0 1]
buildings [0 1]
electricity [0 1]
tools [0 1]
hospitals [0 1]
shops [0 1]
aid_centers [0 1]
other_infrastructure [0 1]
weather_related [0 1]
floods [0 1]
storm [0 1]
fire [0 1]
earthquake [0 1]
cold [0 1]
other_weather [0 1]
direct_report [0 1]


#### Note: "related" has three classes and "child_alone" has one class.

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [10]:
# drop the original categories column from `df`
df = df.merge(df_form, left_on='id', right_on='id')
df = df.drop(['categories'], axis=1)
df.head()

,id,message,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [11]:
# check number of duplicates
df.duplicated(keep='first').sum()

206

In [12]:
# drop duplicates
df = df.drop_duplicates()

In [13]:
# check number of duplicates
df.duplicated(keep='first').sum()

0

### 7. Fix the Category Faults

In [14]:
# sample some 2 entries
print(df[df['related']==2]['message'].sample(10))

6518     Midi Estimene:rue Lambert Prolongee and rue Me...
25385    Mali's former President Amadou Toumani Touré -...
7235     annot ni batiman dada cheri se pwason ni ou ni...
20192    crop, loss of hydro-electric generating capaci...
7743                   NOTES: this message is not complete
7448                     Wesantyahoo.fr.Pepayisenyahoo.fr 
5107     mwen ta renmen gen enfmasyon sou siklyn nan pe...
10299    RT Fleegerian_Akin RT ShanCali Earthquake in c...
12396    TAUSEEF KHAN S/O AKBAR DIN MIANWALI TEH ESSA K...
12365                    .  /    khairo Disctt. Jacobabad.
Name: message, dtype: object


In [15]:
# sample some legit 0 entries
print(df[df['related']==0]['message'].sample(10))

18777    A vaccine for the Influenza A(H1N1) virus will...
1481                         sports information. football.
21920    We must develop new ways of thinking more holi...
26336    Meanwhile, Agritrade [ http://agritrade.cta.in...
11586    RT odrigoBNO: Chile President Bachelet tells r...
19014    That included tightening control over social n...
9488           I don't understand what you are saying me. 
8889     If my heart in your heart it will be one heart...
20682    The democratic party, meanwhile, is more popul...
1014     I'm the former chief secretary of O. N. A. I. .. 
Name: message, dtype: object


In [16]:
# confirm unrelated other classifications for related 2's
print(df[df['related']==2][category_colnames[1:]].sum(axis=0))

request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report             0
dtype: int64


In [17]:
# confirm there's many samples of related 0's
len(df[df['related']==0]), len(df[df['related']==2]), len(df)

(6119, 187, 26180)

In [18]:
# drop the related 2's as likely unrelated and likely illegit
df.drop(df[df['related']==2].index, inplace = True)
len(df)

25993

In [19]:
# confirm no child_alone entries
len(df[df['child_alone']==1])

0

In [20]:
# Split off Validation data for Augmenting purposes

# start with some related zero's
dfv = df[df['related']==0].sample(36).copy()

idx_list = dfv.index.tolist()
# add in some one's for each target category
for cat in category_colnames:
    # manage child alone has no entries
    if cat != 'child_alone':
        df_slice = df[df[cat]==1].copy()
        samp = df_slice.sample()
        while samp.index[0] in dfv.index:
            samp = df_slice.sample() 
        dfv = pd.concat([dfv, samp], axis=0)
    else:
        pass
    
# independant train and validation data 
dft = df[~df.index.isin(dfv.index)].copy()

In [23]:
# Simulate child alone Messages
df_aug = augmenter.BuildCAs(dft)

# sample a 'child alone' entry for validation
samp = df_aug.sample()
dfv = pd.concat([dfv, samp], axis=0)

# Simulate low count class messages
dfa = augmenter.BuildSims(df_aug, dft)

### 8. Pre-clean the message strings  
This uses the imported spell checking cleaner utility

In [24]:
# apply text cleaner
df['message'] = df.message.apply(cleaner.CleanText)
print(df['message'].sample(25))

3210     that a why i need help because i have nothing ...
23441    ten thousand chlorine tablets to purify water ...
20707    six medical doctors participated in overseas t...
0        weather update a cold front from cuba that cou...
4478     i think before school opened we have ask god t...
24194    the challenge for governments with far assista...
4571     someone died in kano saida next to a christ ro...
8656     good morning united nations today is five days...
24420    cereals pulses sugar and highly nutritious cor...
4467     i watch with my eyes while i suffer that the w...
9189                  i want them give us tent in st louis
13626    according to the report the main hazards that ...
20081    it was littered with debris tree trunks muddy ...
22221    post harvest grain losses are assumed at numbe...
22601    she says she has added ground peanuts for prot...
13230    another snow storm in new york help path sokot...
13092    every time i eat an animal shaped chicken nugg.

In [25]:
# related message character = count stats
df['message'].str.len().describe()

count    25993.000000
mean       142.876428
std        207.040570
min          0.000000
25%         73.000000
50%        122.000000
75%        177.000000
max      10753.000000
Name: message, dtype: float64

In [27]:
def Truncate(text, length=501):
    '''
    input string and trim length
    strip and rejoin
    trim to nearest word if too long
    return truncated cleaned string
    '''
    strip = text.rstrip()
    if len(strip) < length:
        clean = ' '.join(strip.split())                  
    else:                
        tokens = strip[:length + 1].split()
        clean = ' '.join(tokens[0:-1])
                         
    return clean

In [28]:
# Apply long message truncation and verify
df['message'] = df['message'].apply(Truncate)
df['message'].str.len().describe()

count    25993.000000
mean       134.902551
std         80.391795
min          0.000000
25%         73.000000
50%        122.000000
75%        177.000000
max        500.000000
Name: message, dtype: float64

In [29]:
### drop rows with tiny messages of questionable utility
df.drop(df[df['message'].str.len()<=27].index, inplace = True)
len(df)

25944

In [30]:
df['message'].str.len().describe()

count    25944.000000
mean       135.116636
std         80.315613
min         28.000000
25%         73.000000
50%        122.000000
75%        177.000000
max        500.000000
Name: message, dtype: float64

### 9. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [31]:
# combine original and augmented dataframes
df_all = pd.concat([df, dfa], axis = 0)

# add flags for tracking purposes
df_all['val'] = df_all.index.isin(dfv.index)
df_all['sim'] = df_all.index.isin(dfa.index)

In [32]:
engine = create_engine('sqlite:///DisasterResponse_r8.db')
df_all.to_sql('MessCatRaw', engine, index=False)

In [33]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43944 entries, 0 to 17999
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      43944 non-null  int64 
 1   message                 43944 non-null  object
 2   related                 43944 non-null  int64 
 3   request                 43944 non-null  int64 
 4   offer                   43944 non-null  int64 
 5   aid_related             43944 non-null  int64 
 6   medical_help            43944 non-null  int64 
 7   medical_products        43944 non-null  int64 
 8   search_and_rescue       43944 non-null  int64 
 9   security                43944 non-null  int64 
 10  military                43944 non-null  int64 
 11  child_alone             43944 non-null  int64 
 12  water                   43944 non-null  int64 
 13  food                    43944 non-null  int64 
 14  shelter                 43944 non-null  int64 
 15  cl

### 10. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [39]:
import etl_pipeline

In [40]:
!python etl_pipeline.py messages.csv categories.csv DisasterResponse_r9.db

Loading data...
    MESSAGES: messages.csv
    CATEGORIES: categories.csv
Formatting data...
Simulating data...
Cleaning data...
Trimming data...
Joining data...
Saving data...
    DATABASE: DisasterResponse_r9.db
Cleaned data saved to database!


In [41]:
conn = sqlite3.connect('DisasterResponse_r9.db')

In [42]:
df_read = pd.read_sql('SELECT * FROM MessCatRaw', con = conn)

In [43]:
df_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43944 entries, 0 to 43943
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      43944 non-null  int64 
 1   message                 43944 non-null  object
 2   related                 43944 non-null  int64 
 3   request                 43944 non-null  int64 
 4   offer                   43944 non-null  int64 
 5   aid_related             43944 non-null  int64 
 6   medical_help            43944 non-null  int64 
 7   medical_products        43944 non-null  int64 
 8   search_and_rescue       43944 non-null  int64 
 9   security                43944 non-null  int64 
 10  military                43944 non-null  int64 
 11  child_alone             43944 non-null  int64 
 12  water                   43944 non-null  int64 
 13  food                    43944 non-null  int64 
 14  shelter                 43944 non-null  int64 
 15  cl

In [44]:
df_read.columns

Index(['id', 'message', 'related', 'request', 'offer', 'aid_related',
       'medical_help', 'medical_products', 'search_and_rescue', 'security',
       'military', 'child_alone', 'water', 'food', 'shelter', 'clothing',
       'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report', 'val', 'sim'],
      dtype='object')